<a href="https://colab.research.google.com/github/hamish-haggerty/cancer-proj/blob/main/cancer-proj/nbs/cancer_dataloading.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# cancer_dataloading

> Helper functions etc to load cancer data. Will also install nbdev etc if in Colab (not sure yet if I need this functionality)

In [ ]:
#| default_exp cancer_dataloading

In [ ]:
#| hide

try:
    from google.colab import drive
    drive.mount('/content/drive')
    !git pull
    !unzip -q "/content/drive/My Drive/archive (1).zip"

    %cd drive/MyDrive/cancer-proj
    #!pip install -e .
    !pip install -qU -e .
    !pip install -qU nbdev
    !nbdev_install_quarto


except ModuleNotFoundError:
    pass

Mounted at /content/drive
fatal: not a git repository (or any of the parent directories): .git
/content/drive/MyDrive/cancer-proj
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 569.1/569.1 KB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.6/41.6 KB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 549.1/549.1 KB 36.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.1/64.1 KB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.1/58.1 KB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.0/79.0 KB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 52.8 MB/s eta 0:00:00
Installing or upgrading quarto -- this requires root access.
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
     

In [ ]:
#| hide

from nbdev.showdoc import *

In [ ]:
#| export

#TODO: Refactor base_rbt library such that can import in one line 

import fastai
from fastai.vision.all import *

from base_rbt.base_model import * 
from base_rbt.base_lf import *
from base_rbt.base_linear import *
from base_rbt.helper import *


In [ ]:
#| export
#colab
colab_train_dir='skin_cancer_ISIC/Train'
colab_test_dir='skin_cancer_ISIC/Test'

In [ ]:
#| export
#local
local_train_dir='/Users/hamishhaggerty/Downloads/skin_cancer_ISIC/Train'
local_test_dir='/Users/hamishhaggerty/Downloads/skin_cancer_ISIC/Test'

In [ ]:
#| hide

try:
    from google.colab import drive
    train_dir = colab_train_dir
    test_dir = colab_test_dir

except ModuleNotFoundError:

    train_dir = local_train_dir
    test_dir = local_test_dir

In [ ]:
#| export

#Seems all we need here is class_names?

def get_file_lists(train_dir):

    #train
    class_names0 = os.listdir(train_dir)
    class_names = sorted(class_names0)
    num_class = len(class_names)
    image_files = [[os.path.join(train_dir, class_name, x) 
                for x in os.listdir(os.path.join(train_dir, class_name))] 
                for class_name in class_names]

    image_file_list = []
    image_label_list = []
    for i, class_name in enumerate(class_names):
        image_file_list.extend(image_files[i])
        image_label_list.extend([i] * len(image_files[i]))
    num_total = len(image_label_list)

    return {'image_file_list':image_file_list, 'image_label_list':image_label_list, 'num_total':num_total, 'num_class':num_class, 'class_names':class_names}


In [ ]:
#| export

#Helper functions to extract class names from the filenames
import re
def extract_text(string):
    # Use the compile method to create a RegexObject
    pattern = re.compile(r'/Train/(.*?)/ISIC')

    # Use the search method of the RegexObject to find the pattern in the string
    match = pattern.search(string)

    # If a match is found, return the matched text
    if match:
        return match.group(1)
    # If no match is found, return None
    else:
        
        return None

def label_func(x): return extract_text(x.as_posix())

def get_difference(x1, x2):
    return list(set(x1) - set(x2))


In [ ]:
#| export

def get_fnames_dict(train_dir,test_dir,class_names):
    "get dictionary of fnames"

        #files names
    fnames = get_image_files(train_dir)

    #Extract training set
    max_num =100 #maximum number of samples in each class
    count_dict = {i:0 for i in class_names}
    fnames_train = []
    for i in fnames:
        #st=extract_text(i.as_posix())
        st=label_func(i)
        
        if count_dict[st]<max_num: #no more than 100 samples per category
            fnames_train.append(i)
            count_dict[st]+=1
                    
    #We further partition fnames_train into a tune-valiation set
    count_dict2 = {i:0 for i in class_names}
    fnames_tune = []
    for i in fnames_train:
        st = label_func(i)
        if count_dict2[st] < 0.8*count_dict[st]:
            fnames_tune.append(i)
            count_dict2[st]+=1
            

    fnames_valid = get_difference(fnames_train,fnames_tune)

    fnames_test = get_difference(fnames,fnames_train) + get_image_files(test_dir)

    return {'fnames':fnames,'fnames_train':fnames_train,'fnames_tune':fnames_tune,
            'fnames_valid':fnames_valid,
            'fnames_test':fnames_test
            }



In [ ]:
#| export

def get_data_dict(fnames_dict,train_dir,test_dir, #basic stuff needed
                  device,bs_val,bs=256,bs_tune=256,size=128,n_in=3 #hyperparameters
                 ):
        "get dictionary of data"

        #fnames = fnames_dict['fnames']
        fnames_train = fnames_dict['fnames_train']
        fnames_tune = fnames_dict['fnames_tune']
        fnames_valid = fnames_dict['fnames_valid']
        #fnames_test = fnames_dict['fnames_test']

        item_tfms = [Resize(size)]

        dls_train  = ImageDataLoaders.from_path_func(train_dir, fnames_train, label_func,
                                        bs=bs,
                                        item_tfms=item_tfms,
                                        valid_pct=0,
                                        device=device,
                                        num_workers=12*(device=='cuda')
                                        )
        x,y = dls_train.one_batch()

        dls_tune = ImageDataLoaders.from_path_func(train_dir, fnames_tune, label_func,
                                        bs=bs_tune,
                                        item_tfms=item_tfms,
                                        valid_pct=0,
                                        device=device,
                                        shuffle=True,
                                        num_workers=12*(device=='cuda'),
                                        )
        xtune,ytune = dls_tune.one_batch()

        dls_valid  = ImageDataLoaders.from_path_func(train_dir, fnames_valid, label_func,
                                        bs=bs_val,
                                        item_tfms=item_tfms,
                                        valid_pct=0,
                                        num_workers=12*(device=='cuda')
                                        )
        
        xval,yval = dls_valid.one_batch()

        vocab = dls_valid.vocab

        #return the dls etc
        return {'dls_train':dls_train,'dls_tune':dls_tune,'dls_valid':dls_valid,
                'x':x,'y':y,'xval':xval,'yval':yval,'xtune':xtune,'ytune':ytune,
                'vocab':vocab
                }


In [ ]:
#| export

def get_fnames_dls_dict(train_dir,test_dir,
                        device,bs_val,bs=256,bs_tune=256,size=128,n_in=3,
                        ):

    "Wrapper that returns a dictionary with the fnames, dls etc"

    #do stuff

    class_names = get_file_lists(train_dir)['class_names']
    
    fnames_dict = get_fnames_dict(train_dir,test_dir,class_names)

    data_dict = get_data_dict(fnames_dict,train_dir,test_dir, #basic stuff needed
                  device,bs_val,bs=256,bs_tune=256,size=128,n_in=3 #hyperparameters
                 )

    d = {**fnames_dict,**data_dict}
    
    return d



In [ ]:
#| hide

#How we want to use

#define general hps
device ='cuda' if torch.cuda.is_available() else 'cpu'
bs=256
bs_tune=256
size=128
bs_val=174
#can define other hyperparameters here

#get the data dictionary
data_dict = get_fnames_dls_dict(train_dir=train_dir,test_dir=test_dir,
                    device=device,bs_val=bs_val,bs=bs,bs_tune=bs_tune,size=size,n_in=3)

#get the dataloaders and other stuff 
dls_train,dls_tune,dls_valid = data_dict['dls_train'],data_dict['dls_tune'],data_dict['dls_valid']
x,y = data_dict['x'],data_dict['y']
xval,yval = data_dict['xval'],data_dict['yval']
xtune,ytune = data_dict['xtune'],data_dict['ytune']
vocab = data_dict['vocab']

#If we want to write some tests (make sure the data is same every time etc):
fnames,fnames_train,fnames_tune,fnames_valid,fnames_test = data_dict['fnames'],data_dict['fnames_train'],data_dict['fnames_tune'],data_dict['fnames_valid'],data_dict['fnames_test']

In [ ]:
#| hide

#Some simple tests

#test shapes
test_eq(x.shape[0],bs)
test_eq(xtune.shape[0],bs_tune)
test_eq(xval.shape[0],bs_val)

#test yval has same number of instances of each class each time...
if yval.shape[0] == 174:
    test_eq(yval.sum().item(),688.)

#TODO: Write tests that check that the fnames are the same every time


In [ ]:
#| export

def get_resnet_encoder(model,n_in=3):
    model = create_body(model, n_in=n_in, pretrained=False, cut=len(list(model.children()))-1)
    model.add_module('flatten', torch.nn.Flatten())
    return model

def create_model(which_model=None,n_in=3):

    #pretrained=True if 'which_model' in ['bt_pretrain', 'supervised_pretrain'] else False

    if which_model == 'bt_pretrain': model = torch.hub.load('facebookresearch/barlowtwins:main', 'resnet50')
    
    elif which_model == 'no_pretrain': model = resnet50()

    elif which_model == 'supervised_pretrain': model = resnet50(weights='IMAGENET1K_V2')

    #ignore the 'pretrained=False' argument here. Just means we use the weights above 
    #(which themselves are either pretrained or not)
    encoder = get_resnet_encoder(model)

    model = create_barlow_twins_model(encoder, hidden_size=ps,projection_size=ps,nlayers=3)

    if device == 'cuda':
        model.cuda()
        encoder.cuda()


    return model,encoder

In [ ]:
#| hide

#test

ps=8192
device=device=default_device().type
cuda=(device=='cuda')

model,encoder = create_model(which_model='bt_pretrain')
for p in model.parameters():
    test_eq(p.requires_grad,True)

#| hide

def sum_params(model):

    s=0
    for p in model.parameters():
        s+=p.sum()
    return s.item()


def run_tests():

    #Test that shape of encoder is 2048
    _,e = create_model(which_model='bt_pretrain')
    test_eq(e(x).shape[1],2048) 

    #Test that pretrained models (encoders) have the same weights (roughyl) each time
    _,e = create_model(which_model='bt_pretrain')
    test_eq((sum_params(e)-175864.4062)<0.001,True)

    _,e = create_model(which_model='supervised_pretrain')
    test_eq((sum_params(e)-39438.5078125)<0.001,True)

    #Test that non-pretrained model has diff (i.e. random) weights each time
    _,e1 = create_model(which_model='no_pretrain')
    _,e2 = create_model(which_model='no_pretrain')
    test_ne(sum_params(e1),sum_params(e2))

run_tests()
# if device == 'cuda':
#     run_tests()

Using cache found in /root/.cache/torch/hub/facebookresearch_barlowtwins_main
Using cache found in /root/.cache/torch/hub/facebookresearch_barlowtwins_main
Using cache found in /root/.cache/torch/hub/facebookresearch_barlowtwins_main
Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

In [1]:
#| hide

print('hello world')

hello world


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()